**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Oct 24 04:42:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15316, done.
remote: Counting objects: 100% (15316/15316), done.
remote: Compressing objects: 100% (4581/4581), done.
remote: Total 15316 (delta 10408), reused 15315 (delta 10408), pack-reused 0
Receiving objects: 100% (15316/15316), 13.72 MiB | 17.51 MiB/s, done.
Resolving deltas: 100% (10408/10408), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Fish" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-10-24 04:45:31--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  22.7MB/s    in 7.5s    

2021-10-24 04:45:39 (20.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/vidframe1.jpg  
  inflating: data/obj/vidframe1.txt  
  inflating: data/obj/vidframe18.jpg  
  inflating: data/obj/vidframe18.txt  
  inflating: data/obj/vidframe19.jpg  
  inflating: data/obj/vidframe19.txt  
  inflating: data/obj/vidframe2.jpg  
  inflating: data/obj/vidframe2.txt  
  inflating: data/obj/vidframe20.jpg  
  inflating: data/obj/vidframe20.txt  
  inflating: data/obj/vidframe21.jpg  
  inflating: data/obj/vidframe21.txt  
  inflating: data/obj/vidframe22.jpg  
  inflating: data/obj/vidframe22.txt  
  inflating: data/obj/vidframe23.jpg  
  inflating: data/obj/vidframe23.txt  
  inflating: data/obj/vidframe24.jpg  
  inflating: data/obj/vidframe24.txt  
  inflating: data/obj/vidframe25.jpg  
  inflating: data/obj/vidframe25.txt  
  inflating: data/obj/vidframe26.jpg  
  inflating: data/obj/vidframe26.txt  
  inflating: data/obj/vidframe27.jpg  
  inflating: data/obj/vidframe27

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.755078 0.890278 0.058594 0.075000

['15', '0.755078', '0.890278', '0.058594', '0.075000']
0 data/obj/vidframe56.txt
0 0.755078 0.890278 0.058594 0.075000
15 0.447656 0.745833 0.023438 0.080556

['15', '0.447656', '0.745833', '0.023438', '0.080556']
1 data/obj/vidframe57.txt
0 0.447656 0.745833 0.023438 0.080556
15 0.700000 0.646528 0.067187 0.112500

['15', '0.700000', '0.646528', '0.067187', '0.112500']
2 data/obj/vidframe25.txt
0 0.700000 0.646528 0.067187 0.112500
15 0.469531 0.790278 0.018750 0.091667

['15', '0.469531', '0.790278', '0.018750', '0.091667']
3 data/obj/vidframe72.txt
0 0.469531 0.790278 0.018750 0.091667
15 0.566016 0.747917 0.066406 0.068056

['15', '0.566016', '0.747917', '0.066406', '0.068056']
4 data/obj/vidframe79.txt
0 0.566016 0.747917 0.066406 0.068056
15 0.486719 0.709722 0.059375 0.063889

['15', '0.486719', '0.709722', '0.059375', '0.063889']
5 data/obj/vidframe61.txt
0 0.486719 0.709722 0.059375 0.063889
15 0.542969 0.759028 0.075000 0.065278

['15',

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/vidframe26.jpg', 'data/obj/vidframe55.jpg', 'data/obj/vidframe72.jpg', 'data/obj/vidframe5.jpg', 'data/obj/vidframe51.jpg', 'data/obj/vidframe78.jpg', 'data/obj/vidframe48.jpg', 'data/obj/vidframe66.jpg', 'data/obj/vidframe65.jpg', 'data/obj/vidframe19.jpg', 'data/obj/vidframe18.jpg', 'data/obj/vidframe39.jpg', 'data/obj/vidframe58.jpg', 'data/obj/vidframe63.jpg', 'data/obj/vidframe9.jpg', 'data/obj/vidframe53.jpg', 'data/obj/vidframe49.jpg', 'data/obj/vidframe46.jpg', 'data/obj/vidframe73.jpg', 'data/obj/vidframe37.jpg', 'data/obj/vidframe62.jpg', 'data/obj/vidframe23.jpg', 'data/obj/vidframe8.jpg', 'data/obj/vidframe64.jpg', 'data/obj/vidframe31.jpg', 'data/obj/vidframe44.jpg', 'data/obj/vidframe1.jpg', 'data/obj/vidframe77.jpg', 'data/obj/vidframe4.jpg', 'data/obj/vidframe7.jpg', 'data/obj/vidframe61.jpg', 'data/obj/vidframe56.jpg', 'data/obj/vidframe36.jpg', 'data/obj/vidframe76.jpg', 'data/obj/vidframe79.jpg', 'data/obj/vidframe25.jpg', 'data/obj/vidframe28.jpg', 'data/

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
#Create testing.txt file
file = open("data/test.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.580199), count: 4, class_loss = 0.914871, iou_loss = 0.677998, total_loss = 1.592869 
 total_bbox = 49371, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.003709, iou_loss = 0.000000, total_loss = 0.003709 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.671000), count: 3, class_loss = 0.730809, iou_loss = 0.233133, total_loss = 0.963941 
 total_bbox = 49374, rewritten_bbox = 0.000000 % 

 852: 0.436780, 0.479486 avg loss, 0.000527 rate, 11.217706 seconds, 54528 images, 12.602016 hours left
Loaded: 0.000064 seconds
Can't open label file. (This can be normal on